In [16]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse

# Function to download an asset
def download_asset(asset_url):
    # Check if the URL is protocol-relative (starts with //)
    if asset_url.startswith('//'):
        asset_url = f'https:{asset_url}'  # Add the 'https' scheme

    asset_response = requests.get(asset_url)
    if asset_response.status_code == 200:
        print("asset_response" , asset_url)
        filename = os.path.join('ucsd_website', os.path.basename(urlparse(asset_url).path))
        with open(filename, 'wb') as file:
            file.write(asset_response.content)

# Function to download CSS assets (images referenced in CSS)
def download_css_assets(css_url):
    css_response = requests.get(css_url)
    if css_response.status_code == 200:
        css_content = css_response.text
        image_urls = re.findall(r'url\(["\'](.*?)["\']\)', css_content)

        for image_url in image_urls:
            image_url = urljoin(css_url, image_url)  # Construct the absolute image URL
            download_asset(image_url)

# Define the URL of the website you want to download
url = 'https://ucsd.edu'

# Create a directory to save the website content
if not os.path.exists('ucsd_website'):
    os.makedirs('ucsd_website')

# Send an HTTP GET request to the website
response = requests.get(url)

if response.status_code == 200:
    # Parse the HTML content of the website using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Save the main HTML content
    with open('ucsd_website/index.html', 'w', encoding='utf-8') as file:
        file.write(str(soup))

    # Find and download CSS, JS, and image files
    assets = soup.find_all(['link', 'script', 'img'])
    print(len(assets))
    for idx, asset in enumerate(assets):
        try :
            print("Tring for asset", asset, idx)
            if asset.has_attr('href'):
                asset_url = urljoin(url, asset['href'])  # Construct the absolute URL
                download_asset(asset_url)
            elif asset.has_attr('src'):
                asset_url = urljoin(url, asset['src'])  # Construct the absolute URL
                download_asset(asset_url)
        except Exception as e:
            print("Exception for asset", asset)
    
    css_links = soup.find_all('link', rel='stylesheet')
    for css_link in css_links:
        try:
            css_url = urljoin(url, css_link['href'])  # Construct the absolute URL
            download_css_assets(css_url)
        except Exception as e:
            print("Exception for css_link", css_link)

    print(f'Website downloaded and saved in the "ucsd_website" directory.')
else:
    print(f'Failed to download the website. Status code: {response.status_code}')




50
Tring for asset <link href="favicon.ico" rel="icon"/> 0
asset_response https://ucsd.edu/favicon.ico
Tring for asset <link href="https://www.ucsd.edu/_resources/css/vendor/brix_sans.css" rel="stylesheet" type="text/css"/> 1
asset_response https://www.ucsd.edu/_resources/css/vendor/brix_sans.css
Tring for asset <link href="https://www.ucsd.edu/_resources/css/vendor/refrigerator_deluxe.css" rel="stylesheet" type="text/css"/> 2
asset_response https://www.ucsd.edu/_resources/css/vendor/refrigerator_deluxe.css
Tring for asset <link href="https://www.ucsd.edu/_resources/css/styles.css" rel="stylesheet"/> 3
asset_response https://www.ucsd.edu/_resources/css/styles.css
Tring for asset <link href="_resources/css/custom-img.css" rel="stylesheet"/> 4
asset_response https://ucsd.edu/_resources/css/custom-img.css
Tring for asset <link href="https://www.ucsd.edu/_resources/css/custom.css" rel="stylesheet"/> 5
asset_response https://www.ucsd.edu/_resources/css/custom.css
Tring for asset <link href=